# Capstone Projekt Rossmann

# Reports

## Individual Store Reports

### Definition of Information and KPIs to be reported

#### Information

- Period
- Store ID
- Store Type
- Assortment
- Competition Distance
- Competition Open Since
- Promo2Since
- Promo2Interval


#### KPIs

- Line plot with sales
- Line plot with SalesPerOpenDay
- Line plot with sales per customer
- Line plot with customers
- Line plot with CustomersPerOpenDay
- Line plot with IsPromo
- Line plot with Promo2Active
- Line plot with SchoolHoliday
- Line plot with NumStateHoliday  
- Rolling mean
- Compare with other storeTypes and assortments
- weekly and monthly

#### Input Fields

- StoreId
- StartDate
- EndDate

### Store Report

#### Input and pre setup

In [47]:
import pandas as pd
import numpy as np
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from pandas.api.types import infer_dtype
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import warnings
pd.set_option('display.max_columns', None)

In [48]:
# Input fields to define the store and the date range
StoreId = 955
StartDate = "2014-01-01"
EndDate = "2015-12-31"

StartDate = datetime.strptime(StartDate, "%Y-%m-%d")
EndDate = datetime.strptime(EndDate, "%Y-%m-%d")

In [49]:
df = pd.read_csv('weekly_sales_with_store_info.csv', parse_dates=['Date'])

In [50]:
print(df.info())
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150525 entries, 0 to 150524
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Store                      150525 non-null  int64         
 1   Date                       150525 non-null  datetime64[ns]
 2   CW                         150525 non-null  int64         
 3   Month                      150525 non-null  int64         
 4   Year                       150525 non-null  int64         
 5   DayOfWeek                  150525 non-null  int64         
 6   Sales                      150525 non-null  int64         
 7   SalesPerCustomer           145809 non-null  float64       
 8   SalesPerOpenDay            145815 non-null  float64       
 9   Customers                  150525 non-null  int64         
 10  CustomersPerOpenDay        145815 non-null  float64       
 11  Open                       150525 non-null  int64   

,Store,Date,CW,Month,Year,DayOfWeek,Sales,SalesPerCustomer,SalesPerOpenDay,Customers,CustomersPerOpenDay,Open,Promo,IsPromo,StateHoliday,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,IsCompetition,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Promo2Member,Promo2Active
15992,119,2014-03-16,11,3,2014,6,28845,7.994734,4807.500000,3608,601.333333,6,0,0,0,0,0,0,0,a,c,2100.0,2.0,2010.0,1,0,NaN,NaN,NaN,0,0
66014,489,2015-08-02,31,8,2015,6,51792,13.561665,10358.400000,3819,763.800000,5,5,1,0,0,5,1,0,a,a,14960.0,11.0,2013.0,1,1,37.0,2009.0,"Jan,Apr,Jul,Oct",1,0
99155,735,2014-04-06,14,4,2014,6,34875,10.096989,5812.500000,3454,575.666667,6,5,1,0,0,0,0,0,d,c,1920.0,4.0,2005.0,1,0,NaN,NaN,NaN,0,0
142626,1057,2014-04-13,15,4,2014,6,31096,9.354994,5182.666667,3324,554.000000,6,0,0,0,0,0,0,0,d,c,3230.0,11.0,2011.0,1,0,NaN,NaN,NaN,0,0
120986,897,2013-07-07,27,7,2013,6,18025,9.876712,3004.166667,1825,304.166667,6,5,1,0,0,0,0,0,c,c,3290.0,1.0,2007.0,1,1,5.0,2013.0,"Feb,May,Aug,Nov",1,0


In [51]:
# Generate dataframes based on input fields
df_store = df[(df['Store'] == StoreId) & (df['Date'] >= StartDate) & (df['Date'] <= EndDate)]
StoreType = df_store['StoreType'].iloc[0]
Assortment = df_store['Assortment'].iloc[0]

# Dataframe for comparison with all stores with the same StoreType and Assortment exept the selected one
df_store_compare = df[(df['Store'] != StoreId) & (df['Date'] >= StartDate) & (df['Date'] <= EndDate) & (df['StoreType'] == StoreType) & (df['Assortment'] == Assortment)]
df_store.head()

,Store,Date,CW,Month,Year,DayOfWeek,Sales,SalesPerCustomer,SalesPerOpenDay,Customers,CustomersPerOpenDay,Open,Promo,IsPromo,StateHoliday,IsStateHoliday,SchoolHoliday,IsSchoolHoliday,NumStateHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,IsCompetition,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Promo2Member,Promo2Active
128842,955,2014-01-05,1,1,2014,6,20192,9.547045,4038.400000,2115,423.000000,5,0,0,a,1,5,1,1,d,c,1690.0,7.0,2009.0,1,1,36.0,2013.0,"Mar,Jun,Sept,Dec",1,0
128843,955,2014-01-12,2,1,2014,6,33204,11.309264,5534.000000,2936,489.333333,6,5,1,0,0,2,1,0,d,c,1690.0,7.0,2009.0,1,1,36.0,2013.0,"Mar,Jun,Sept,Dec",1,0
128844,955,2014-01-19,3,1,2014,6,20049,8.898802,3341.500000,2253,375.500000,6,0,0,0,0,0,0,0,d,c,1690.0,7.0,2009.0,1,1,36.0,2013.0,"Mar,Jun,Sept,Dec",1,0
128845,955,2014-01-26,4,1,2014,6,31043,10.922942,5173.833333,2842,473.666667,6,5,1,0,0,0,0,0,d,c,1690.0,7.0,2009.0,1,1,36.0,2013.0,"Mar,Jun,Sept,Dec",1,0
128846,955,2014-02-02,5,2,2014,6,23487,9.501214,3914.500000,2472,412.000000,6,0,0,0,0,0,0,0,d,c,1690.0,7.0,2009.0,1,1,36.0,2013.0,"Mar,Jun,Sept,Dec",1,0


In [52]:
# Create a mean dataframe out of all stores with the same StoreType and Assortment
columns_to_average = ['Sales', 'SalesPerOpenDay', 'SalesPerCustomer', 'Customers', 
                      'CustomersPerOpenDay', 'IsPromo', 'Promo2Active', 
                      'SchoolHoliday', 'NumStateHoliday']

# Group df_store_compare by 'Date' and calculate the mean of the specified columns
mean_df = df_store_compare.groupby('Date')[columns_to_average].mean().reset_index()
mean_df.head()

,Date,Sales,SalesPerOpenDay,SalesPerCustomer,Customers,CustomersPerOpenDay,IsPromo,Promo2Active,SchoolHoliday,NumStateHoliday
0,2014-01-05,32500.908676,6500.181735,10.839721,3032.178082,606.435616,0.0,0.301370,5.310502,1.0000
1,2014-01-12,46006.324201,8147.683866,12.050595,3876.652968,683.655556,1.0,0.301370,1.406393,0.3379
2,2014-01-19,32867.497717,5477.916286,10.416119,3202.150685,533.691781,0.0,0.301370,0.000000,0.0000
3,2014-01-26,45061.470320,7510.245053,12.104361,3758.821918,626.470320,1.0,0.301370,0.000000,0.0000
4,2014-02-02,34766.484018,5794.414003,10.856693,3246.369863,541.061644,0.0,0.109589,0.018265,0.0000


#### Store Report - Weekly Basis

In [53]:
# Print report header

print(f"Period: {StartDate.strftime('%Y-%m-%d')} to {EndDate.strftime('%Y-%m-%d')}")
print(f"Store ID: {StoreId}")
print(f"Store Type: {StoreType}")
print(f"Assortment: {Assortment}")

if df_store['IsCompetition'].iloc[0] == 1:
	print(f"Competition Distance: {int(df_store['CompetitionDistance'].iloc[0])}")
	CompetitionOpenSince = datetime(int(df_store['CompetitionOpenSinceYear'].iloc[0]), int(df_store['CompetitionOpenSinceMonth'].iloc[0]), 1)
	print(f"Competition Open Since: {CompetitionOpenSince.strftime('%Y-%m-%d')}")
else:
	print("Competition: None")

if df_store['Promo2'].iloc[0] == 1:
	Promo2Since = datetime.strptime(f"{int(df_store['Promo2SinceYear'].iloc[0])} {int(df_store['Promo2SinceWeek'].iloc[0])} 1", '%G %V %u').date()
	print(f"Promo2Since: {Promo2Since.strftime('%Y-%m-%d')}")
	print(f"PromoInterval: {df_store['PromoInterval'].iloc[0]}")
else:
	print("Promo2: None")



Period: 2014-01-01 to 2015-12-31
Store ID: 955
Store Type: d
Assortment: c
Competition Distance: 1690
Competition Open Since: 2009-07-01
Promo2Since: 2013-09-02
PromoInterval: Mar,Jun,Sept,Dec


In [54]:
# plot the weekly overview

rolling_mean_window = 12

# Suppress all warnings in the current cell
with warnings.catch_warnings():
	warnings.simplefilter("ignore")
	# Calculate the rolling mean for the last X weeks for each column
	df_store['Sales_rolling_mean'] = df_store['Sales'].rolling(window=rolling_mean_window).mean()
	df_store['SalesPerOpenDay_rolling_mean'] = df_store['SalesPerOpenDay'].rolling(window=rolling_mean_window).mean()
	df_store['SalesPerCustomer_rolling_mean'] = df_store['SalesPerCustomer'].rolling(window=rolling_mean_window).mean()
	df_store['Customers_rolling_mean'] = df_store['Customers'].rolling(window=rolling_mean_window).mean()
	df_store['CustomersPerOpenDay_rolling_mean'] = df_store['CustomersPerOpenDay'].rolling(window=rolling_mean_window).mean()

# Create a subplot grid
fig = make_subplots(rows=9, cols=1, subplot_titles=('Sales', 'SalesPerOpenDay', 'SalesPerCustomer', 'Customers', 'CustomersPerOpenDay', 'IsPromo', 'Promo2Active', 'SchoolHoliday', 'NumStateHoliday'))

# Add the original and rolling mean plots to the subplot grid
# Sales
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['Sales'], mode='lines+markers', name='Sales', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['Sales_rolling_mean'], mode='lines', name=f'Sales {rolling_mean_window}-Week Rolling Mean', line=dict(dash='dot', color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['Sales'], mode='lines+markers', name='Sales compare', line=dict(dash='dash', color='green')), row=1, col=1)

# SalesPerOpenDay
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['SalesPerOpenDay'], mode='lines+markers', name='SalesPerOpenDay', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['SalesPerOpenDay_rolling_mean'], mode='lines', name=f'SalesPerOpenDay {rolling_mean_window}-Week Rolling Mean', line=dict(dash='dot', color='red')), row=2, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['SalesPerOpenDay'], mode='lines+markers', name='SalesPerOpenDay compare', line=dict(dash='dash', color='green')), row=2, col=1)

# SalesPerCustomer
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['SalesPerCustomer'], mode='lines+markers', name='SalesPerCustomer', line=dict(color='blue')), row=3, col=1)
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['SalesPerCustomer_rolling_mean'], mode='lines', name=f'SalesPerCustomer {rolling_mean_window}-Week Rolling Mean', line=dict(dash='dot', color='red')), row=3, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['SalesPerCustomer'], mode='lines+markers', name='SalesPerCustomer compare', line=dict(dash='dash', color='green')), row=3, col=1)

# Customers
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['Customers'], mode='lines+markers', name='Customers', line=dict(color='blue')), row=4, col=1)
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['Customers_rolling_mean'], mode='lines', name=f'Customers {rolling_mean_window}-Week Rolling Mean', line=dict(dash='dot', color='red')), row=4, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['Customers'], mode='lines+markers', name='Customers compare', line=dict(dash='dash', color='green')), row=4, col=1)

# CustomersPerOpenDay
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['CustomersPerOpenDay'], mode='lines+markers', name='CustomersPerOpenDay', line=dict(color='blue')), row=5, col=1)
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['CustomersPerOpenDay_rolling_mean'], mode='lines', name=f'CustomersPerOpenDay {rolling_mean_window}-Week Rolling Mean', line=dict(dash='dot', color='red')), row=5, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['CustomersPerOpenDay'], mode='lines+markers', name='CustomersPerOpenDay compare', line=dict(dash='dash', color='green')), row=5, col=1)

# IsPromo
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['IsPromo'], mode='lines+markers', name='IsPromo', line=dict(color='blue')), row=6, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['IsPromo'], mode='lines+markers', name='IsPromo compare', line=dict(dash='dash', color='green')), row=6, col=1)

# Promo2Active
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['Promo2Active'], mode='lines+markers', name='Promo2Active', line=dict(color='blue')), row=7, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['Promo2Active'], mode='lines+markers', name='Promo2Active compare', line=dict(dash='dash', color='green')), row=7, col=1)

# SchoolHoliday
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['SchoolHoliday'], mode='lines+markers', name='Amount of school holidays', line=dict(color='blue')), row=8, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['SchoolHoliday'], mode='lines+markers', name='Amount of school holidays compare', line=dict(dash='dash', color='green')), row=8, col=1)

# NumStateHoliday
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['NumStateHoliday'], mode='lines+markers', name='Amount of state holiday', line=dict(color='blue')), row=9, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['NumStateHoliday'], mode='lines+markers', name='Amount of state holiday compare', line=dict(dash='dash', color='green')), row=9, col=1)

# Update layout (set layout properties and show the legend)
fig.update_layout(height=1800, width=1400, title_text=f"Store {StoreId} Weekly Overview", showlegend=True)

# Show the figure
print( '\033[1m' + 'Info: The compare numbers are the average of the same store type and assortment'+ '\033[0m')
fig.show()


Info: The compare numbers are the average of the same store type and assortment


#### Store Report - Monthly Basis

In [55]:
# Prepare monthly dataframe

monthly_summary_df_store = df_store.groupby(['Year', 'Month']).agg(
    {
        'Sales': 'sum',
        'SalesPerOpenDay': 'mean',
        'SalesPerCustomer': 'mean',
        'Customers': 'sum',
        'CustomersPerOpenDay': 'mean',
        'IsPromo': 'sum', 
        'Promo2Active': 'sum',
        'SchoolHoliday': 'sum',
        'NumStateHoliday': 'sum'
    }
).reset_index()

# Add a new 'Date' column representing the first day of each month
monthly_summary_df_store['Date'] = pd.to_datetime(monthly_summary_df_store['Year'].astype(str) + '-' + monthly_summary_df_store['Month'].astype(str) + '-01')

# Reorder columns
monthly_summary_df_store = monthly_summary_df_store[['Date', 'Year', 'Month', 'Sales', 'SalesPerOpenDay', 'SalesPerCustomer', 'Customers', 'CustomersPerOpenDay', 'IsPromo', 'Promo2Active', 'SchoolHoliday', 'NumStateHoliday']]

monthly_summary_df_store.head()

,Date,Year,Month,Sales,SalesPerOpenDay,SalesPerCustomer,Customers,CustomersPerOpenDay,IsPromo,Promo2Active,SchoolHoliday,NumStateHoliday
0,2014-01-01,2014,1,104488,4521.933333,10.169513,10146,440.375000,2,0,7,1
1,2014-02-01,2014,2,115103,4795.958333,10.222102,11168,465.333333,2,0,0,0
2,2014-03-01,2014,3,138076,4602.533333,10.075565,13656,455.200000,2,5,0,0
3,2014-04-01,2014,4,105226,4804.025000,9.994680,10425,475.091667,2,0,10,2
4,2014-05-01,2014,5,120818,5319.683333,10.345011,11609,509.491667,3,0,0,1


In [56]:
# plot the monthly overview with comparison

df_store = monthly_summary_df_store

df_store_compare = df[(df['Store'] != StoreId) & (df['Date'] >= StartDate) & (df['Date'] <= EndDate) & (df['StoreType'] == StoreType) & (df['Assortment'] == Assortment)]


monthly_summary_df_store_all = df_store_compare.groupby(['Year', 'Month']).agg(
    {
        'Sales': 'mean',
        'SalesPerOpenDay': 'mean',
        'SalesPerCustomer': 'mean',
        'Customers': 'mean',
        'CustomersPerOpenDay': 'mean',
        'IsPromo': 'mean', 
        'Promo2Active': 'mean',
        'SchoolHoliday': 'mean',
        'NumStateHoliday': 'mean'
    }
).reset_index()

# Add a new 'Date' column representing the first day of each month
monthly_summary_df_store_all['Date'] = pd.to_datetime(monthly_summary_df_store_all['Year'].astype(str) + '-' + monthly_summary_df_store_all['Month'].astype(str) + '-01')

# Reorder columns
monthly_summary_df_store_all = monthly_summary_df_store_all[['Date', 'Year', 'Month', 'Sales', 'SalesPerOpenDay', 'SalesPerCustomer', 'Customers', 'CustomersPerOpenDay', 'IsPromo', 'Promo2Active', 'SchoolHoliday', 'NumStateHoliday']]
mean_df = monthly_summary_df_store_all


rolling_mean_window = 3

# Suppress all warnings in the current cell
with warnings.catch_warnings():
	warnings.simplefilter("ignore")
	# Calculate the rolling mean for the last X Months for each column
	df_store['Sales_rolling_mean'] = df_store['Sales'].rolling(window=rolling_mean_window).mean()
	df_store['SalesPerOpenDay_rolling_mean'] = df_store['SalesPerOpenDay'].rolling(window=rolling_mean_window).mean()
	df_store['SalesPerCustomer_rolling_mean'] = df_store['SalesPerCustomer'].rolling(window=rolling_mean_window).mean()
	df_store['Customers_rolling_mean'] = df_store['Customers'].rolling(window=rolling_mean_window).mean()
	df_store['CustomersPerOpenDay_rolling_mean'] = df_store['CustomersPerOpenDay'].rolling(window=rolling_mean_window).mean()

# Create a subplot grid
fig = make_subplots(rows=9, cols=1, subplot_titles=('Sales', 'SalesPerOpenDay', 'SalesPerCustomer', 'Customers', 'CustomersPerOpenDay', 'IsPromo', 'Promo2Active', 'SchoolHoliday', 'NumStateHoliday'))

# Add the original and rolling mean plots to the subplot grid
# Sales
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['Sales'], mode='lines+markers', name='Sales', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['Sales_rolling_mean'], mode='lines', name=f'Sales {rolling_mean_window}-Month Rolling Mean', line=dict(dash='dot', color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['Sales'], mode='lines+markers', name='Sales compare', line=dict(dash='dash', color='green')), row=1, col=1)

# SalesPerOpenDay
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['SalesPerOpenDay'], mode='lines+markers', name='SalesPerOpenDay', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['SalesPerOpenDay_rolling_mean'], mode='lines', name=f'SalesPerOpenDay {rolling_mean_window}-Month Rolling Mean', line=dict(dash='dot', color='red')), row=2, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['SalesPerOpenDay'], mode='lines+markers', name='SalesPerOpenDay compare', line=dict(dash='dash', color='green')), row=2, col=1)

# SalesPerCustomer
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['SalesPerCustomer'], mode='lines+markers', name='SalesPerCustomer', line=dict(color='blue')), row=3, col=1)
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['SalesPerCustomer_rolling_mean'], mode='lines', name=f'SalesPerCustomer {rolling_mean_window}-Month Rolling Mean', line=dict(dash='dot', color='red')), row=3, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['SalesPerCustomer'], mode='lines+markers', name='SalesPerCustomer compare', line=dict(dash='dash', color='green')), row=3, col=1)

# Customers
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['Customers'], mode='lines+markers', name='Customers', line=dict(color='blue')), row=4, col=1)
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['Customers_rolling_mean'], mode='lines', name=f'Customers {rolling_mean_window}-Month Rolling Mean', line=dict(dash='dot', color='red')), row=4, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['Customers'], mode='lines+markers', name='Customers compare', line=dict(dash='dash', color='green')), row=4, col=1)

# CustomersPerOpenDay
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['CustomersPerOpenDay'], mode='lines+markers', name='CustomersPerOpenDay', line=dict(color='blue')), row=5, col=1)
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['CustomersPerOpenDay_rolling_mean'], mode='lines', name=f'CustomersPerOpenDay {rolling_mean_window}-Month Rolling Mean', line=dict(dash='dot', color='red')), row=5, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['CustomersPerOpenDay'], mode='lines+markers', name='CustomersPerOpenDay compare', line=dict(dash='dash', color='green')), row=5, col=1)

# IsPromo
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['IsPromo'], mode='lines+markers', name='IsPromo', line=dict(color='blue')), row=6, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['IsPromo'], mode='lines+markers', name='IsPromo compare', line=dict(dash='dash', color='green')), row=6, col=1)

# Promo2Active
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['Promo2Active'], mode='lines+markers', name='Promo2Active', line=dict(color='blue')), row=7, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['Promo2Active'], mode='lines+markers', name='Promo2Active compare', line=dict(dash='dash', color='green')), row=7, col=1)

# SchoolHoliday
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['SchoolHoliday'], mode='lines+markers', name='Amount of school holidays', line=dict(color='blue')), row=8, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['SchoolHoliday'], mode='lines+markers', name='Amount of school holidays compare', line=dict(dash='dash', color='green')), row=8, col=1)

# NumStateHoliday
fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['NumStateHoliday'], mode='lines+markers', name='Amount of state holiday', line=dict(color='blue')), row=9, col=1)
fig.add_trace(go.Scatter(x=mean_df['Date'], y=mean_df['NumStateHoliday'], mode='lines+markers', name='Amount of state holiday compare', line=dict(dash='dash', color='green')), row=9, col=1)

# Update layout (set layout properties and show the legend)
fig.update_layout(height=1800, width=1400, title_text=f"Store {StoreId} Monthly Overview", showlegend=True)

# Show the figure
print( '\033[1m' + 'Info: The compare numbers are the average of the same store type and assortment'+ '\033[0m')
fig.show()


Info: The compare numbers are the average of the same store type and assortment


In [57]:
# plot the monthly overview

# Create a subplot grid
fig = make_subplots(rows=9, cols=1, subplot_titles=('Sales', 'SalesPerOpenDay', 'SalesPerCustomer', 'Customers', 'CustomersPerOpenDay', 'IsPromo', 'Promo2Active', 'SchoolHoliday', 'NumStateHoliday'))

# Add each plot to the subplot grid with specific names
fig.add_trace(go.Scatter(x=monthly_summary_df_store['Date'], y=monthly_summary_df_store['Sales'], mode='lines+markers', name='Sales'), row=1, col=1)
fig.add_trace(go.Scatter(x=monthly_summary_df_store['Date'], y=monthly_summary_df_store['SalesPerOpenDay'], mode='lines+markers', name='SalesPerOpenDay'), row=2, col=1)
fig.add_trace(go.Scatter(x=monthly_summary_df_store['Date'], y=monthly_summary_df_store['SalesPerCustomer'], mode='lines+markers', name='SalesPerCustomer'), row=3, col=1)
fig.add_trace(go.Scatter(x=monthly_summary_df_store['Date'], y=monthly_summary_df_store['Customers'], mode='lines+markers', name='Customers'), row=4, col=1)
fig.add_trace(go.Scatter(x=monthly_summary_df_store['Date'], y=monthly_summary_df_store['CustomersPerOpenDay'], mode='lines+markers', name='CustomersPerOpenDay'), row=5, col=1)
fig.add_trace(go.Scatter(x=monthly_summary_df_store['Date'], y=monthly_summary_df_store['IsPromo'], mode='lines+markers', name='IsPromo'), row=6, col=1)
fig.add_trace(go.Scatter(x=monthly_summary_df_store['Date'], y=monthly_summary_df_store['Promo2Active'], mode='lines+markers', name='Promo2Active'), row=7, col=1)
fig.add_trace(go.Scatter(x=monthly_summary_df_store['Date'], y=monthly_summary_df_store['SchoolHoliday'], mode='lines+markers', name='Amount of school holidays'), row=8, col=1)
fig.add_trace(go.Scatter(x=monthly_summary_df_store['Date'], y=monthly_summary_df_store['NumStateHoliday'], mode='lines+markers', name='Amount of state holiday'), row=9, col=1)

# Update layout (set layout properties and hide the legend)
fig.update_layout(height=1600, width=1100, title_text=f"Store {StoreId} Monthly Overview", showlegend=False)

# Show the figure
fig.show()


## Overall Report

### Definition of Information and KPIs to be reported

#### Information

- Period
- Number of Stores
- Number of Stores in each Store Type
- Number of Stores in each Assortment
- Number of StoreType in each Assortment
- Number of Assortment in each StoreType
- Number of Stores with Promo
- Number of Stores without Promo
- Number of Stores with Promo2
- Number of Stores without Promo2
- Number of Stores with Competition



#### KPIs

- Line plot with sales for each week for all stores
- Line plot with sales for each week for each store type
- Line plot with sales for each week for each assortment

- Line plot with SalesPerOpenDay for each week for all stores
- Line plot with SalesPerOpenDay for each week for each store type
- Line plot with SalesPerOpenDay for each week for each assortment

- Line plot with sales per customer for each week for all stores
- Line plot with sales per customer for each week for each store type
- Line plot with sales per customer for each week for each assortment

- Line plot with customers for each week for all stores
- Line plot with customers for each week for each store type
- Line plot with customers for each week for each assortment

- Line plot with CustomersPerOpenDay for each week for all stores
- Line plot with CustomersPerOpenDay for each week for each store type
- Line plot with CustomersPerOpenDay for each week for each assortment



- Bar plot with amount of stores that have promo and no promo
- Bar plot with amount of stores that have promo2 and no promo2

#### Input Fields

- StartDate
- EndDate
- (StoreType)
- (Assortment)

### Input and pre setup

In [58]:
# Input fields to define the store and the date range

StartDate = "2013-01-01"
EndDate = "2014-12-31"
StartDate = datetime.strptime(StartDate, "%Y-%m-%d")
EndDate = datetime.strptime(EndDate, "%Y-%m-%d")

In [59]:
df_all_stores = df[(df['Date'] >= StartDate) & (df['Date'] <= EndDate)]

In [60]:
monthly_summary = df_all_stores.groupby(['Year', 'Month']).agg(
    {
        'Sales': 'sum',
        'SalesPerOpenDay': 'mean',
        'SalesPerCustomer': 'mean',
        'Customers': 'sum',
        'CustomersPerOpenDay': 'mean',
        'IsPromo': 'mean', 
        'Promo2Active': 'sum',
        'SchoolHoliday': 'mean',
        'NumStateHoliday': 'mean'
    }
).reset_index()

# Add a new 'Date' column representing the first day of each month
monthly_summary['Date'] = pd.to_datetime(monthly_summary['Year'].astype(str) + '-' + monthly_summary['Month'].astype(str) + '-01')

# Reorder columns
monthly_summary = monthly_summary[['Date', 'Year', 'Month', 'Sales', 'SalesPerOpenDay', 'SalesPerCustomer', 'Customers', 'CustomersPerOpenDay', 'IsPromo', 'Promo2Active', 'SchoolHoliday', 'NumStateHoliday']]

monthly_summary.head()



,Date,Year,Month,Sales,SalesPerOpenDay,SalesPerCustomer,Customers,CustomersPerOpenDay,IsPromo,Promo2Active,SchoolHoliday,NumStateHoliday
0,2013-01-01,2013,1,155822491,6322.836886,9.273179,17471528,709.885841,0.5,912,1.141928,0.319058
1,2013-02-01,2013,2,171439913,6427.544368,9.342311,19190788,717.751004,0.5,392,0.604933,0.000000
2,2013-03-01,2013,3,225584447,7033.858730,9.451784,24932037,773.210937,0.6,339,1.020448,0.200000
3,2013-04-01,2013,4,162826157,6324.883565,9.090353,18786879,728.266352,0.5,952,1.192601,0.250000
4,2013-05-01,2013,5,164459743,6956.306618,9.295173,18548458,781.232953,0.5,408,0.562556,0.750000


In [61]:
# Print report header

#- Period
print(f"Period: {StartDate.strftime('%Y-%m-%d')} to {EndDate.strftime('%Y-%m-%d')}"	)

#- Number of Stores
print(f"Number of Stores: {df_all_stores['Store'].nunique()}")

#- Number of Stores in each Store Type
print("Number of Stores in each Store Type:", df_all_stores.groupby('StoreType')['Store'].nunique())
store_counts = df_all_stores.groupby('StoreType')['Store'].nunique().reset_index(name='NumberOfStores')
fig = px.pie(store_counts, names='StoreType', values='NumberOfStores', title='Number of Stores in each Store Type')
fig.update_traces(hovertemplate='%{label}: %{value} (<b>%{percent}</b>)')
fig.show()

#- Number of Stores in each Assortment
print("Number of Stores in each Assortment:", df_all_stores.groupby('Assortment')['Store'].nunique())
assortment_counts = df_all_stores.groupby('Assortment')['Store'].nunique().reset_index(name='NumberOfStores')
fig = px.pie(assortment_counts, names='Assortment', values='NumberOfStores', title='Number of Stores in each Assortment')
fig.update_traces(hovertemplate='%{label}: %{value} (<b>%{percent}</b>)')
fig.show()

#- Number of Stores with Promo2
print(f"Number of Stores with Promo2: {df_all_stores.groupby('Promo2')['Store'].nunique().loc[1]}")

#- Number of Stores without Promo2
print(f"Number of Stores without Promo2: {df_all_stores.groupby('Promo2')['Store'].nunique().loc[0]}")

#- Number of Stores with Competition
print(f"Number of Stores with Competition: {df_all_stores.groupby('IsCompetition')['Store'].nunique().loc[1]}")

#- Number of Stores without Competition
print(f"Number of Stores without Competition: {df_all_stores.groupby('IsCompetition')['Store'].nunique().loc[0]}")

Period: 2013-01-01 to 2014-12-31
Number of Stores: 1115
Number of Stores in each Store Type: StoreType
a    602
b     17
c    148
d    348
Name: Store, dtype: int64


Number of Stores in each Assortment: Assortment
a    593
b      9
c    513
Name: Store, dtype: int64


Number of Stores with Promo2: 571
Number of Stores without Promo2: 544
Number of Stores with Competition: 723
Number of Stores without Competition: 545


In [62]:
# Plot monthly overview

rolling_mean_window = 3
import warnings

# Suppress all warnings in the current cell
with warnings.catch_warnings():
	warnings.simplefilter("ignore")
	# Calculate the rolling mean for the last X Months for each column
	monthly_summary['Sales_rolling_mean'] = monthly_summary['Sales'].rolling(window=rolling_mean_window).mean()
	monthly_summary['SalesPerOpenDay_rolling_mean'] = monthly_summary['SalesPerOpenDay'].rolling(window=rolling_mean_window).mean()
	monthly_summary['SalesPerCustomer_rolling_mean'] = monthly_summary['SalesPerCustomer'].rolling(window=rolling_mean_window).mean()
	monthly_summary['Customers_rolling_mean'] = monthly_summary['Customers'].rolling(window=rolling_mean_window).mean()
	monthly_summary['CustomersPerOpenDay_rolling_mean'] = monthly_summary['CustomersPerOpenDay'].rolling(window=rolling_mean_window).mean()

# Create a subplot grid
fig = make_subplots(rows=10, cols=1, subplot_titles=('Sales', 'SalesPerOpenDay', 'SalesPerCustomer', 'Customers', 'CustomersPerOpenDay', 'IsPromo', 'Promo2Active', 'SchoolHoliday', 'NumStateHoliday'))

# Add the original and rolling mean plots to the subplot grid
# Sales
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['Sales'], mode='lines+markers', name='Sales', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['Sales_rolling_mean'], mode='lines', name=f'Sales {rolling_mean_window}-Month Rolling Mean', line=dict(dash='dot', color='red')), row=1, col=1)
    
# SalesPerOpenDay
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['SalesPerOpenDay'], mode='lines+markers', name='SalesPerOpenDay', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['SalesPerOpenDay_rolling_mean'], mode='lines', name=f'SalesPerOpenDay {rolling_mean_window}-Month Rolling Mean', line=dict(dash='dot', color='red')), row=2, col=1)

# SalesPerCustomer
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['SalesPerCustomer'], mode='lines+markers', name='SalesPerCustomer', line=dict(color='blue')), row=3, col=1)
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['SalesPerCustomer_rolling_mean'], mode='lines', name=f'SalesPerCustomer {rolling_mean_window}-Month Rolling Mean', line=dict(dash='dot', color='red')), row=3, col=1)

# Customers
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['Customers'], mode='lines+markers', name='Customers', line=dict(color='blue')), row=4, col=1)
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['Customers_rolling_mean'], mode='lines', name=f'Customers {rolling_mean_window}-Month Rolling Mean', line=dict(dash='dot', color='red')), row=4, col=1)

# CustomersPerOpenDay
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['CustomersPerOpenDay'], mode='lines+markers', name='CustomersPerOpenDay', line=dict(color='blue')), row=5, col=1)
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['CustomersPerOpenDay_rolling_mean'], mode='lines', name=f'CustomersPerOpenDay {rolling_mean_window}-Month Rolling Mean', line=dict(dash='dot', color='red')), row=5, col=1)

# IsPromo
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['IsPromo'], mode='lines+markers', name='IsPromo', line=dict(color='blue')), row=6, col=1)

# Promo2Active
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['Promo2Active'], mode='lines+markers', name='Promo2Active', line=dict(color='blue')), row=7, col=1)

# SchoolHoliday
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['SchoolHoliday'], mode='lines+markers', name='Amount of school holidays', line=dict(color='blue')), row=8, col=1)

# NumStateHoliday
fig.add_trace(go.Scatter(x=monthly_summary['Date'], y=monthly_summary['NumStateHoliday'], mode='lines+markers', name='Amount of state holiday', line=dict(color='blue')), row=9, col=1)

# Update layout (set layout properties and show the legend)
fig.update_layout(height=1800, width=1400, title_text=f"Overall Monthly Overview", showlegend=True)

# Show the figure
fig.show()